In [22]:
import pandas as pd
import sqlite3

# Establish connection
con = sqlite3.connect("my_data1.db")

# Load data into the table
df_csv = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df_csv.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

# Create the cleaned table (removing null dates)
con.execute("DROP TABLE IF EXISTS SPACEXTABLE")
con.execute("CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL")

In [23]:
# Task 1: Unique launch sites
print("Task 1:")
display(pd.read_sql_query("SELECT DISTINCT Launch_Site FROM SPACEXTABLE", con))

# Task 2: 5 records starting with 'CCA'
print("\nTask 2:")
display(pd.read_sql_query("SELECT * FROM SPACEXTABLE WHERE Launch_Site LIKE 'CCA%' LIMIT 5", con))

# Task 3: Total payload mass for NASA (CRS)
print("\nTask 3:")
display(pd.read_sql_query("SELECT SUM(PAYLOAD_MASS__KG_) AS Total_Payload_Mass FROM SPACEXTABLE WHERE Customer = 'NASA (CRS)'", con))

# Task 4: Average payload for F9 v1.1
print("\nTask 4:")
display(pd.read_sql_query("SELECT AVG(PAYLOAD_MASS__KG_) AS Average_Payload_Mass FROM SPACEXTABLE WHERE Booster_Version = 'F9 v1.1'", con))

# Task 5: First successful ground pad landing
print("\nTask 5:")
display(pd.read_sql_query("SELECT MIN(Date) FROM SPACEXTABLE WHERE Landing_Outcome = 'Success (ground pad)'", con))

Task 1:


,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40



Task 2:


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt



Task 3:


,Total_Payload_Mass
0,45596



Task 4:


,Average_Payload_Mass
0,2928.4



Task 5:


,MIN(Date)
0,2015-12-22


In [24]:
# Task 6: Successful drone ship boosters (4000kg - 6000kg)
print("\nTask 6:")
display(pd.read_sql_query("SELECT Booster_Version FROM SPACEXTABLE WHERE Landing_Outcome = 'Success (drone ship)' AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000", con))

# Task 7: Total successful and failure outcomes
print("\nTask 7:")
display(pd.read_sql_query("SELECT Mission_Outcome, COUNT(*) AS Total_Count FROM SPACEXTABLE GROUP BY Mission_Outcome", con))

# Task 8: Boosters with the maximum payload mass
print("\nTask 8:")
display(pd.read_sql_query("SELECT DISTINCT Booster_Version FROM SPACEXTABLE WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE)", con))

# Task 9: Drone ship failures in 2015
print("\nTask 9:")
display(pd.read_sql_query("SELECT substr(Date, 6, 2) AS Month, Landing_Outcome, Booster_Version, Launch_Site FROM SPACEXTABLE WHERE substr(Date, 1, 4) = '2015' AND Landing_Outcome = 'Failure (drone ship)'", con))

# Task 10: Rank outcomes between 2010-06-04 and 2017-03-20
print("\nTask 10:")
query_10 = """
SELECT Landing_Outcome, COUNT(*) AS Outcome_Count 
FROM SPACEXTABLE 
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20' 
GROUP BY Landing_Outcome 
ORDER BY Outcome_Count DESC
"""
display(pd.read_sql_query(query_10, con))


Task 6:


,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2



Task 7:


,Mission_Outcome,Total_Count
0,Failure (in flight),1
1,Success,98
2,Success,1
3,Success (payload status unclear),1



Task 8:


,Booster_Version
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6



Task 9:


,Month,Landing_Outcome,Booster_Version,Launch_Site
0,01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40



Task 10:


,Landing_Outcome,Outcome_Count
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1
